In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from datetime import datetime,date,timedelta
import pickle

In [2]:
ordir={"North":0.0,
    "NNE":22.5,
    "NE": 45.0,
    "ENE":67.5,
    "East":90.0,
    "ESE":112.5,
    "SE":135.0,
    "SSE":157.5,
    "South":180,
    "SSW":202.5,
    "SW":225.0,
    "WSW":247.5,
    "West":270.0,
    "WNW":292.5,
    "NW":315.0,
    "NNW":337.5}

In [3]:
def dailywind(d):
    wdate = d
    url = f"https://www.wunderground.com/dashboard/pws/KCASANTA106/table/{wdate}/{wdate}/daily"
    html = urlopen(url)
    soup = BeautifulSoup(html)

    headers = [th.getText() for th in soup.findAll('thead')[3].findAll('th')]
    rows = soup.findAll('tbody')[3].findAll('tr')
    hourly_data = [[td.getText() for td in rows[i].findAll('td')] for i in range(len(rows))]

    df_weather = pd.DataFrame(hourly_data,columns=headers)
    df_wind = df_weather[['Time','Temperature','Wind','Speed','Gust']]
    df_wind['Date'] = wdate
    df_wind['Orient']= df_wind['Wind'].map(ordir)

    df_wind = df_wind[['Date','Time','Temperature','Wind','Orient','Speed','Gust']]
    df_wind.columns = ['Date', 'Time','Temp','Wind','Orient', 'Speed', 'Gust']

    return df_wind

In [4]:
now = datetime.now().date()
sdate= datetime.now().date()-timedelta(days=365*2)
datediff = (now-sdate).days
print(f"# full days to pull between start date ({sdate}) and now ({now}): {datediff}")

# full days to pull between start date (2020-10-04) and now (2022-10-04): 730


In [ ]:
nested_dfs=[]
for i in range(datediff):
    weatherdate = sdate + timedelta(days = i)
    nested_dfs.append(dailywind(weatherdate))

In [6]:
sdatenew=nested_dfs[-1].Date.iloc[0]+timedelta(days=1)
datediffnew=(now-sdatenew).days
print(f"interrupted before pulling: {sdatenew}")
print(f"days pulled: {len(nested_dfs)}")
print(f"days remaining: {datediffnew}")

interrupted before pulling: 2021-08-22
days pulled: 322
days remaining: 408


In [45]:
for i in range(datediffnew):
    weatherdate = sdatenew + timedelta(days = i)
    nested_dfs.append(dailywind(weatherdate))

In [49]:
allwind = nested_dfs[0]
for i in range(1,len(nested_dfs)):
    allwind = pd.concat([allwind,nested_dfs[i]],axis=0)
allwind['Speed'] = allwind['Speed'].str[:-5].astype(float)
allwind['Gust'][allwind['Gust']=='--'] = '0.0 °mph'
allwind['Gust'] = allwind['Gust'].str[:-5].astype(float)
allwind['Temp'] = allwind['Temp'].str[:-3].astype(float)
allwind['Month'] = pd.DatetimeIndex(allwind['Date']).month_name().str[:3]

In [62]:
allwind.to_pickle(f'brommer_raw_{sdate}_{now-timedelta(days=1)}.pkl')

In [64]:
allwind.to_csv(f'brommer_raw_{sdate}_{now-timedelta(days=1)}.csv')